# Import Libraries

In [ ]:
# Import Libraries

import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import skimage 
import os
import gc
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import scanpy.external as sce


# Export Irf1 Stat1 expression across all spots

In [ ]:
# Export Irf1 and Stat1 expression

adata = sc.read_h5ad("Objects/adata_filtered_Celltypes.h5ad")
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

genes_of_interest = ['Irf1', 'Stat1']
missing = [g for g in genes_of_interest if g not in adata.var_names]
if missing:
    raise ValueError(f"Missing genes in adata: {missing}")

expr = adata[:, genes_of_interest].X.toarray() if hasattr(adata[:, genes_of_interest].X, "toarray") else adata[:, genes_of_interest].X

df = pd.DataFrame(expr, columns=genes_of_interest)
df['cell'] = adata.obs_names
df = df[['cell', 'Irf1', 'Stat1']]


df.to_csv("Irf1_Stat1_normalized.csv", index=False)



# Gene Signatures LPS / IRF1 / Stat1

In [ ]:
# Load gene signature CSV
adata = sc.read_h5ad("Objects/adata_filtered_Celltypes.h5ad")
signatures_df = pd.read_csv("Signatures/GeneSignature_df_LPSIrf1Stat1.csv")
print(signatures_df.head())  

# Set dirs
base_dir = "GeneSignatures/AppliedSignatures"
gene_lists_dir = r"GeneSignatures/Genes_perSig_Actual"

os.makedirs(gene_lists_dir, exist_ok=True)
os.makedirs(base_dir, exist_ok=True)

# Pull unique organs
organs = signatures_df['tissue'].unique()
print(f"All organs are: {organs}")

# Loop through organs
for organ in organs:
    print(f"Processing organ: {organ}")

    organ_adata = adata[adata.obs['Organ'] == organ].copy()

    # Normalize organ subset
    sc.pp.normalize_total(organ_adata, target_sum=1e4, inplace=True)
    sc.pp.log1p(organ_adata)

    # Get relevant group/direction combinations
    combos = signatures_df[signatures_df['tissue'] == organ][['group', 'direction']].drop_duplicates()
    print(f"relevant group/direction combos are: {combos}")

    # Apply each signature-direction combo
    for _, row in combos.iterrows():
        group = row['group']
        direction = row['direction']
        print(f"row is: {row}")
        print(f"group is: {group}")
        print(f"direction is: {direction}")

        genes = signatures_df[
            (signatures_df['tissue'] == organ) &
            (signatures_df['group'] == group) &
            (signatures_df['direction'] == direction)
        ]['gene'].tolist()

        print(f"Genes examined for this combo are: {genes}")

        score_label = f"{organ}_{group}_{direction}_score"
        print(f"score_label is: {score_label}")

        genes_present = [gene for gene in genes if gene in organ_adata.var_names]
        genes_missing = [gene for gene in genes if gene not in organ_adata.var_names]

        # Save each list as CSV
        pd.DataFrame(genes_present, columns=['gene']).to_csv(
            os.path.join(gene_lists_dir, f"{organ}_{score_label}_genes_present.csv"),
            index=False
        )

        pd.DataFrame(genes_missing, columns=['gene']).to_csv(
            os.path.join(gene_lists_dir, f"{organ}_{score_label}_genes_missing.csv"),
            index=False
        )

        if len(genes_present) == 0:
            print(f"WARNING: No valid genes present for {score_label}, skipping.")
            continue  # Skip scoring if no valid genes        
        
        # Compute gene scores
        sc.tl.score_genes(organ_adata, gene_list=genes_present, score_name=score_label)

    # Prepare and export scores as CSV
    score_cols = [f"{organ}_{row['group']}_{row['direction']}_score" for _, row in combos.iterrows()]
    print(f"columsns that will be exported from object metadata are: {score_cols}")
    score_df = organ_adata.obs[score_cols].copy()

    # Export CSV per organ
    score_df.to_csv(f"{base_dir}/{organ}_SignatureScores.csv")
    print(f"This df will be stored as: {base_dir}/{organ}_SignatureScores.csv")

    print(f"Completed: {organ} scores exported.")
